<a href="https://colab.research.google.com/github/marcomag416/MLDL/blob/main/DeepLabV2_metrics_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Preparation code

In [1]:
#install fvcore
!pip install -U fvcore

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 1.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for fvcore: filename=fvcore-0.1.5.post20221221-py3-none-any.whl size=61400 sha256=bbff272131b5ad1e8cb478b31b66b368f2dfb4a1b54ffb8e451504b5cb9f904b
  Stored in directory: /root/.cache/pip/wheels/01/c0/af/77c1cf53a1be9e42a52b48e5af2169d40ec2e89f7362489dd0
  Created wheel for iopath: filename=iopath-0.1.10-py3-none-any.whl size=31532 sha256=9cf68a97b1de611838216f03773ea9dd2d61fdf25cdca1342fd5f3ba642cdce0
  Stored in directory: /root/.cache/pip/wheels/9a/a3/b6/ac0fcd1b4ed5cfeb3db92e6a0e476cfd48ed0df92b91080c1d
Successfully built fvcore iopath


In [2]:
import gdown

# Download DeepLabv2 model from TA's repository
url = 'https://github.com/Gabrysse/MLDL2024_project1/raw/master/models/deeplabv2/deeplabv2.py'
output_path = 'deeplabv2.py'
gdown.download(url, output_path, quiet=False)

Downloading...
From: https://github.com/Gabrysse/MLDL2024_project1/raw/master/models/deeplabv2/deeplabv2.py
To: /content/deeplabv2.py
6.78kB [00:00, 6.17MB/s]                   


'deeplabv2.py'

In [3]:
# Download the DeepLabv2 pretrained weights
url = 'https://drive.google.com/uc?id=1ZX0UCXvJwqd2uBGCX7LI2n-DfMg3t74v'
weight_path = './deeplab_resnet_pretrained_imagenet.pth'
gdown.download(url, weight_path, quiet=False)

Downloading...
From (original): https://drive.google.com/uc?id=1ZX0UCXvJwqd2uBGCX7LI2n-DfMg3t74v
From (redirected): https://drive.google.com/uc?id=1ZX0UCXvJwqd2uBGCX7LI2n-DfMg3t74v&confirm=t&uuid=85e1313e-212d-433e-b1c1-b3dda27d7e9a
To: /content/deeplab_resnet_pretrained_imagenet.pth
100%|██████████| 177M/177M [00:07<00:00, 23.2MB/s]


'./deeplab_resnet_pretrained_imagenet.pth'

# Metrics computation
### FLOPS and parameters

In [4]:
#compute number of flops
from fvcore.nn import FlopCountAnalysis, flop_count_table

import deeplabv2
from torch import nn
import torch
from torch.optim.lr_scheduler import PolynomialLR
import numpy as np
import os

# Set the manual seeds
torch.manual_seed(42)
torch.cuda.manual_seed(42)

device = "cuda" if torch.cuda.is_available() else "cpu"

# -----------------------------
# Initizialize your model here

# Set CUDA_LAUNCH_BLOCKING environment variable
os.environ['CUDA_LAUNCH_BLOCKING']="1"
os.environ['TORCH_USE_CUDA_DSA'] = "1"

model = deeplabv2.get_deeplab_v2(num_classes=19,pretrain_model_path=weight_path).to(device)

# -----------------------------

height = 512
width = 1024
image = torch.randn((1, 3, height, width)).to(device)

flops = FlopCountAnalysis(model, image)
print(flop_count_table(flops))

Deeplab pretraining loading...
| module                         | #parameters or shape   | #flops     |
|:-------------------------------|:-----------------------|:-----------|
| model                          | 43.901M                | 0.376T     |
|  conv1                         |  9.408K                |  1.233G    |
|   conv1.weight                 |   (64, 3, 7, 7)        |            |
|  bn1                           |  0.128K                |  41.943M   |
|   bn1.weight                   |   (64,)                |            |
|   bn1.bias                     |   (64,)                |            |
|  layer1                        |  0.216M                |  7.295G    |
|   layer1.0                     |   75.008K              |   2.55G    |
|    layer1.0.conv1              |    4.096K              |    0.136G  |
|    layer1.0.bn1                |    0.128K              |    10.609M |
|    layer1.0.conv2              |    36.864K             |    1.222G  |
|    layer1.0.bn2   

### Latency and FPS

In [5]:
import time
from tqdm.auto import tqdm

print(f"Using {device} device")

iters = 1000
latency = [0]*iters

for i in tqdm(range(iters)):
  start = time.time()
  out = model(image)
  end = time.time()

  latency[i] = end-start

latency = np.array(latency)
fps = 1 / latency

print(f"Mean latency: {latency.mean()} s \nStd latency: {latency.std()} s \nMean FPS: {fps.mean()}\nStd FPS: {fps.std()}")

Using cuda device


  0%|          | 0/1000 [00:00<?, ?it/s]

Mean latency: 0.28605155420303346 s 
Std latency: 0.013511220214174424 s 
Mean FPS: 3.5019946952376646
Std FPS: 0.13255138447129639
